## 仅供交叉验证 前馈神经网络（NNAR）-按趋势分类

In [1]:
import pickle
import numpy as np
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [4]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按趋势划分

In [2]:
def split_data_by_trend(data, targets):
    up_data = []
    down_data = []
    up_target = []
    down_target = []
    
    for i in range(len(data)):
        a, b = np.polyfit(range(len(data[i])), data[i, :, -2].reshape(-1), 1)
        if a > 0:
            up_data.append(data[i])
            up_target.append(targets[i])
        else:
            down_data.append(data[i])
            down_target.append(targets[i])
    return np.array(up_data), np.array(up_target), np.array(down_data), np.array(down_target)

### 构建模型

In [16]:
np.corrcoef([1,2,9,4,9,6], [1,2,3,4,5,6])[0, 1]

0.638885614178983

In [8]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

def build_direct_dnn_model():
    model = keras.models.Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(5))
    
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[]
    }
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2 = split_data_by_trend(X_train, y_train)
        train_xs = [X_train1, X_train2]
        train_ys = [y_train1, y_train2]
        
        X_test1, y_test1, X_test2, y_test2 = split_data_by_trend(X_test, y_test)
        test_xs = [X_test1, X_test2]
        test_ys = [y_test1, y_test2]
        i_s = [1, 2]
        
        # 训练
        for i in range(len(i_s)):
            model = build_direct_dnn_model()
            history = model.fit(train_xs[i], train_ys[i], epochs=100, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

#     for train, test in kfold.split(X, y_cat):
#         model = build_direct_dnn_model()
#         history = model.fit(X[train], y[train], epochs=100, batch_size=16, verbose=1, validation_data=(X[test], y[test]),
#                            callbacks=[
#                                EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
#                            ])

#         y_test = y[test]
#         y_pred = model.predict(X[test]).reshape(y[test].shape)

#         metrics = ['mae', 'rmse','ndcg']
#         for m in metrics:
#             overall, annual = eval_model(m, y_test, y_pred, scaler)
#             overall_metrics[m].append(overall)
#             annual_metrics[m].append(annual)
    
#     return overall_metrics, annual_metrics

In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
#     pickle.dump(metrics, open('mlp_metrics.dict', 'wb'))
    
    return metrics

In [9]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 988 samples, validate on 256 samples
Epoch 1/100
988/988 [==============================] - 1s 803us/sample - loss: 1.1069 - val_loss: 1.0758
Epoch 2/100
988/988 [==============================] - 0s 169us/sample - loss: 1.0539 - val_loss: 1.0848
Epoch 3/100
988/988 [==============================] - 0s 167us/sample - loss: 1.0419 - val_loss: 1.0745
Epoch 4/100
988/988 [==============================] - 0s 165us/sample - loss: 1.0341 - val_loss: 1.0797
Epoch 5/100
988/988 [==============================] - 0s 163us/sample - loss: 1.0255 - val_loss: 1.0769
Epoch 6/100
988/988 [==============================] - 0s 165us/sample - loss: 1.0138 - val_loss: 1.0754
Epoch 7/100
988/988 [==============================] - 0s 165us/sample - loss: 1.0091 - val_loss: 1.0728
Epoch 8/100
988/988 [==============================] - 0s 165us/sample - loss: 1.0000 - val_loss: 1.0689
Epoch 9/100
988/988 

1115/1115 [==============================] - 0s 183us/sample - loss: 0.7381 - val_loss: 0.7770
Epoch 17/100
1115/1115 [==============================] - 0s 180us/sample - loss: 0.7387 - val_loss: 0.7810
Epoch 18/100
1115/1115 [==============================] - 0s 186us/sample - loss: 0.7398 - val_loss: 0.7807
Train on 995 samples, validate on 249 samples
Epoch 1/100
995/995 [==============================] - 1s 1ms/sample - loss: 1.1095 - val_loss: 1.0524
Epoch 2/100
995/995 [==============================] - 0s 176us/sample - loss: 1.0572 - val_loss: 1.0508
Epoch 3/100
995/995 [==============================] - 0s 216us/sample - loss: 1.0430 - val_loss: 1.0563
Epoch 4/100
995/995 [==============================] - 0s 189us/sample - loss: 1.0380 - val_loss: 1.0499
Epoch 5/100
995/995 [==============================] - 0s 188us/sample - loss: 1.0258 - val_loss: 1.0575
Epoch 6/100
995/995 [==============================] - 0s 202us/sample - loss: 1.0215 - val_loss: 1.0535
Epoch 7/100
995

1112/1112 [==============================] - 0s 236us/sample - loss: 0.7611 - val_loss: 0.7275
Epoch 5/100
1112/1112 [==============================] - 0s 239us/sample - loss: 0.7570 - val_loss: 0.7261
Epoch 6/100
1112/1112 [==============================] - 0s 244us/sample - loss: 0.7564 - val_loss: 0.7246
Epoch 7/100
1112/1112 [==============================] - 0s 219us/sample - loss: 0.7538 - val_loss: 0.7275
Epoch 8/100
1112/1112 [==============================] - 0s 220us/sample - loss: 0.7553 - val_loss: 0.7301
Epoch 9/100
1112/1112 [==============================] - 0s 214us/sample - loss: 0.7548 - val_loss: 0.7268
Epoch 10/100
1112/1112 [==============================] - 0s 213us/sample - loss: 0.7542 - val_loss: 0.7251
Epoch 11/100
1112/1112 [==============================] - 0s 209us/sample - loss: 0.7534 - val_loss: 0.7290
Epoch 12/100
1112/1112 [==============================] - 0s 234us/sample - loss: 0.7518 - val_loss: 0.7302
Epoch 13/100
1112/1112 [======================

570/570 [==============================] - 0s 234us/sample - loss: 0.3225 - val_loss: 0.4437
Epoch 13/100
570/570 [==============================] - 0s 241us/sample - loss: 0.3108 - val_loss: 0.4459
Train on 3535 samples, validate on 880 samples
Epoch 1/100
3535/3535 [==============================] - 1s 327us/sample - loss: 0.6695 - val_loss: 0.6499
Epoch 2/100
3535/3535 [==============================] - 1s 171us/sample - loss: 0.6344 - val_loss: 0.6538
Epoch 3/100
3535/3535 [==============================] - 1s 174us/sample - loss: 0.6237 - val_loss: 0.6469
Epoch 4/100
3535/3535 [==============================] - 1s 174us/sample - loss: 0.6142 - val_loss: 0.6458
Epoch 5/100
3535/3535 [==============================] - 1s 174us/sample - loss: 0.6052 - val_loss: 0.6453
Epoch 6/100
3535/3535 [==============================] - 1s 173us/sample - loss: 0.5982 - val_loss: 0.6433
Epoch 7/100
3535/3535 [==============================] - 1s 173us/sample - loss: 0.5904 - val_loss: 0.6516
Epoch

Epoch 11/100
3537/3537 [==============================] - 1s 175us/sample - loss: 0.5573 - val_loss: 0.6697
Epoch 12/100
3537/3537 [==============================] - 1s 173us/sample - loss: 0.5518 - val_loss: 0.6694
Epoch 13/100
3537/3537 [==============================] - 1s 175us/sample - loss: 0.5435 - val_loss: 0.6709
Epoch 14/100
3537/3537 [==============================] - 1s 174us/sample - loss: 0.5309 - val_loss: 0.6718
Epoch 15/100
3537/3537 [==============================] - 1s 176us/sample - loss: 0.5243 - val_loss: 0.6856
Train on 578 samples, validate on 148 samples
Epoch 1/100
578/578 [==============================] - 1s 1ms/sample - loss: 0.5011 - val_loss: 0.4442
Epoch 2/100
578/578 [==============================] - 0s 229us/sample - loss: 0.4229 - val_loss: 0.4426
Epoch 3/100
578/578 [==============================] - 0s 231us/sample - loss: 0.4084 - val_loss: 0.4366
Epoch 4/100
578/578 [==============================] - 0s 240us/sample - loss: 0.3950 - val_loss: 0.4

In [6]:
metrics

{'gene': {'overall': {'mae': 0.7299490896746337,
   'rmse': 1.2918968704150449,
   'ndcg': 0.44835543717483367},
  'annual': {'mae': array([0.41098335, 0.57326106, 0.75468637, 0.88329084, 1.02752384]),
   'rmse': array([0.74612466, 0.97067574, 1.33166402, 1.48635163, 1.66849718]),
   'ndcg': array([0.46505046, 0.36096419, 0.14908492, 0.19261638, 0.15743147])}},
 'transplant': {'overall': {'mae': 0.7718043551135649,
   'rmse': 1.2752554128409044,
   'ndcg': 0.48122490028670706},
  'annual': {'mae': array([0.75218725, 0.77410503, 0.74279975, 0.7759474 , 0.81398235]),
   'rmse': array([1.30078556, 1.29846621, 1.21739863, 1.22831554, 1.31962679]),
   'ndcg': array([0.09773368, 0.07247887, 0.02053076, 0.10604713, 0.11611737])}}}

In [8]:
overall_metrics

{'mae': [0.7786941820597671,
  0.7788960884233309,
  0.7729668810238098,
  0.7533253829680581,
  0.7765691004200828],
 'rmse': [1.27086824299393,
  1.2638911023240982,
  1.247003894606875,
  1.2303769903078015,
  1.3186515565011414],
 'ndcg': [0.06715128409921688,
  0.27366708122800754,
  0.7082721967796911,
  0.7802365522730148,
  0.07262263474874978]}